## Pr?gr?mm?ng?H?m?w?rk 3

Name: S. Kaan Cetindag

I hereby declare that I observed the honour code of the university when preparing the homework.

In [1]:
import csv
import numpy as np
import pandas as pd

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T=[]
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

with open('transitions.csv','rb') as t:
    reader=csv.reader(t,delimiter=',')
    for row in reader:
        T.append(row)
T=np.array(T)
T=T.astype(np.float)

### Question 1

For the first question, I created a cumulative sum of the distribution and drawed from that distribution with an uniform random selection $ \mathcal{U}[0,1] $. The chosen character is the character with the nearest larger probability value to our drawing. The process is repeated by changing the distribution with the previously chosen characters distribution.

In [2]:
def findNearMin(array, val):
    diff=array-val                #difference between each element and given value
    positive=np.where(diff>=0)    #get the index of >0 elements in diff array
    return min(positive[0])       #return the min index that is larger than 0

def generateString(first_letter,N):
    x_prev=first_letter
    string=x_prev

    for i in range(N):
        x_prevRow=np.cumsum(np.asarray(T[letter2idx[x_prev]],dtype=float)) #distribution of the given char as a cum. sum
        w=np.random.rand()                                                 #uniform random variable
        randPick=findNearMin(x_prevRow,w)                                  #get the larger nearest index to the random variable
        x_prev = alphabet[randPick]                                        #append the letter in that index to the string
        string+=x_prev
    print string
    
generateString('.',24)

.ainatithithel.theesonour


### Question 2
To generate the missing letters we use both the previous and next given letters surrounding the missing letters. Even though our model is a Markov(1) model, the use of following given letters is necessary. A simple analogy can be finding a route, knowing the starting point and destination. The process can be formulated as:

For a sequence of $n$ letters, we have 2 letters that encapusulate the string and $n-2$ letters that are unknown.

\begin{equation*}
p(x_{begin+1},x_{end-1}|x_{begin},x_{end})\propto p(x_{end}|x_{end-1})p(x_{end-1}|x_{end-2})....p(x_{begin+2|x_{begin+1}})p(x_{begin+1}|x_{begin})
\end{equation*}

The above equation can be solved by having $T^{n-1}$, getting the row of the end letter distribution and multiplicating it with our beginning letter distribution. The final distribution can be normalized and drawed from similar to question 1.


In [7]:
def generateStringGiven(givenString):
    unknowns=np.empty(0)
    knowns=np.empty(0)
    diffs=np.empty(0)
    prevs=np.empty(0)
    nexts=np.empty(0)
    
    for idx,ch in enumerate(givenString):
        if ch=='_':
            unknowns=np.append(unknowns,idx)
    
    def pickChar(dist):
        dist=np.cumsum(dist, dtype=float)
        w=np.random.rand()
        pick=findNearMin(dist,w)
        return alphabet[pick]
    
    def getPrev(val):
        diff=knowns-val
        negative=np.where(diff<=0)[0]
        if negative.shape[0]==0:
            return None
        else:        
            return int(knowns[max(negative)])
        
    def getNext(val):
        diff=knowns-val
        positive=np.where(diff>=0)[0]
        if positive.shape[0]==0:
            return None
        else:        
            return int(knowns[min(positive)])                              
     
    def generate(previdx,nextidx):
        if previdx==None and nextidx!=None:
            n=1
            prevLetter='.'
            nextLetter=givenString[nextidx]
        elif nextidx==None and previdx!=None:
            n=len(givenString)-previdx-1
            nextLetter='.'
            prevLetter=givenString[previdx]
        else:
            n=nextidx-previdx-1
            prevLetter=givenString[previdx]
            nextLetter=givenString[nextidx]
        
        prev_dist=T[letter2idx[prevLetter]]
        next_dist=np.linalg.matrix_power(T,n).transpose()[letter2idx[nextLetter]]
        final_dist=prev_dist*next_dist
        final_dist/=np.sum(final_dist)
        return pickChar(final_dist)
    
    for j in unknowns:
        j=int(j)
        for idx,ch in enumerate(givenString):
            if ch=='_':
                unknowns=np.append(unknowns,idx)
            else:
                knowns=np.append(knowns,idx)

        prevGiven=getPrev(j)
        nextGiven=getNext(j)
        
        addChar=generate(prevGiven,nextGiven)
        s=list(givenString)
        s[j]=addChar
        givenString=''.join(s)
    print givenString
        
        
for k in range(len(test_strings)):
    print test_strings[k]
    generateStringGiven(test_strings[k])
    print 

th__br__n.f_x.
the.brvin.fex.

_u_st__n_.to_be._nsw_r__
oussthand.toube.answar.h

i__at_._a_h_n_._e_r_i_g
it.at..hathend.fedraing

q___t.___z._____t.__.___.__.
qur.t.mprz.pomy.t.te.fig.fr.



### Question 3
Similar to question 2, but we thake the *argmax()* of the final distribution achieved by matrix power operation.

In [5]:
def generateStringGiven(givenString):
    unknowns=np.empty(0)
    knowns=np.empty(0)
    diffs=np.empty(0)
    prevs=np.empty(0)
    nexts=np.empty(0)
    
    for idx,ch in enumerate(givenString):
        if ch=='_':
            unknowns=np.append(unknowns,idx)
    
    def pickChar(dist):
        dist=np.cumsum(dist, dtype=float)
        w=np.random.rand()
        pick=findNearMin(dist,w)
        return alphabet[pick]
    
    def getPrev(val):
        diff=knowns-val
        negative=np.where(diff<=0)[0]
        if negative.shape[0]==0:
            return None
        else:        
            return int(knowns[max(negative)])
        
    def getNext(val):
        diff=knowns-val
        positive=np.where(diff>=0)[0]
        if positive.shape[0]==0:
            return None
        else:        
            return int(knowns[min(positive)])                              
     
    def generate(previdx,nextidx):
        if previdx==None and nextidx!=None:
            n=1
            prevLetter='.'
            nextLetter=givenString[nextidx]
        elif nextidx==None and previdx!=None:
            n=len(givenString)-previdx-1
            nextLetter='.'
            prevLetter=givenString[previdx]
        else:
            n=nextidx-previdx-1
            prevLetter=givenString[previdx]
            nextLetter=givenString[nextidx]
        
        prev_dist=T[letter2idx[prevLetter]]
        next_dist=np.linalg.matrix_power(T,n).transpose()[letter2idx[nextLetter]]
        final_dist=prev_dist*next_dist
        final_dist/=np.sum(final_dist)
        return alphabet[np.argmax(final_dist)]
    
    for j in unknowns:
        j=int(j)
        for idx,ch in enumerate(givenString):
            if ch=='_':
                unknowns=np.append(unknowns,idx)
            else:
                knowns=np.append(knowns,idx)

        prevGiven=getPrev(j)
        nextGiven=getNext(j)
        
        addChar=generate(prevGiven,nextGiven)
        s=list(givenString)
        s[j]=str(addChar)
        givenString=''.join(s)
    print givenString
        
        
for k in range(len(test_strings)):
    print test_strings[k]
    generateStringGiven(test_strings[k])
    print 
    

th__br__n.f_x.
the.br.an.fex.

_u_st__n_.to_be._nsw_r__
oursthend.to.be.answered

i__at_._a_h_n_._e_r_i_g
in.ath.wathend.he.r.ing

q___t.___z._____t.__.___.__.
qur.t.thiz.the.at.an.the.an.



### Question 4

Models such as Markov(2) and higher would be more suitable for better estimations because dependency to only the first preceeding letter is a poor modeling of languages. This would increase the complexity of the estimation process but in turn also  increase the correct estimation. 

Another alternative could be using well defined $\phi(x_1,x_2)$ functions to implement the forward-backward algorithm.